In [19]:
import numpy as np
import pandas as pd
# Ratio de Sharpe
import yfinance as yf

In [20]:
df = yf.Ticker('NQ=F')
df = df.history(start='2010-01-01', end='2024-01-01')

In [21]:
def backtest_strategy(sma_1, sma_2):
    df['SMA_50'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_200'] = df['Close'].rolling(window=sma_2).mean()

    df['Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'Signal'] = -1

    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    df.dropna(inplace=True)

    # Calcular el ratio de Sharpe
    risk_free_rate = 0.01  # 1% tasa libre de riesgo
    excess_return = df['Strategy_Returns'].mean() - (risk_free_rate / 252)
    sharpe_ratio = excess_return / df['Strategy_Returns'].std()
    
    return sharpe_ratio

### Optimización mediante Grid Search


In [22]:
best_params = None
best_sharpe = -np.inf

In [23]:
for sma_1 in range(10, 100, 10):
    for sma_2 in range(100, 300, 50):
        sharpe = backtest_strategy(sma_1, sma_2)
        print(f'Probando SMA_1: {sma_1}, SMA_2: {sma_2} - Ratio de Sharpe: {sharpe:.2f}')
        
        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_params = (sma_1, sma_2)

Probando SMA_1: 10, SMA_2: 100 - Ratio de Sharpe: -0.00
Probando SMA_1: 10, SMA_2: 150 - Ratio de Sharpe: 0.01
Probando SMA_1: 10, SMA_2: 200 - Ratio de Sharpe: 0.02
Probando SMA_1: 10, SMA_2: 250 - Ratio de Sharpe: 0.02
Probando SMA_1: 20, SMA_2: 100 - Ratio de Sharpe: 0.01
Probando SMA_1: 20, SMA_2: 150 - Ratio de Sharpe: 0.02
Probando SMA_1: 20, SMA_2: 200 - Ratio de Sharpe: 0.02
Probando SMA_1: 20, SMA_2: 250 - Ratio de Sharpe: 0.02
Probando SMA_1: 30, SMA_2: 100 - Ratio de Sharpe: 0.01
Probando SMA_1: 30, SMA_2: 150 - Ratio de Sharpe: 0.04
Probando SMA_1: 30, SMA_2: 200 - Ratio de Sharpe: 0.04
Probando SMA_1: 30, SMA_2: 250 - Ratio de Sharpe: 0.03
Probando SMA_1: 40, SMA_2: 100 - Ratio de Sharpe: 0.01
Probando SMA_1: 40, SMA_2: 150 - Ratio de Sharpe: 0.04
Probando SMA_1: 40, SMA_2: 200 - Ratio de Sharpe: 0.04
Probando SMA_1: 40, SMA_2: 250 - Ratio de Sharpe: 0.04
Probando SMA_1: 50, SMA_2: 100 - Ratio de Sharpe: 0.03
Probando SMA_1: 50, SMA_2: 150 - Ratio de Sharpe: 0.05
Probando 

In [24]:
print(f'Los mejores parámetros son: {best_params} con un Ratio de Sharpe de {best_sharpe:.2f}')

Los mejores parámetros son: (50, 250) con un Ratio de Sharpe de 0.37
